In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Activation, Dropout, Flatten, Concatenate
from keras.layers import Conv2D, Add, Reshape
#combined RPSP
combined_input = Input((12,1,1))
combined_inputt = Reshape([1,12],name='Combined_Input')(combined_input)
x1 = combined_inputt
x1 = Dense(units=1, activation='tanh', name="Densecomb")(x1)
hidden_layers1 = 2
for i in range(hidden_layers1-1):
    x1 = Dense(units=1, activation='tanh', name="Dense1{}".format(i))(x1)
    #x = Dropout(0.2, name='Drop{}'.format(i))(x)
dropped1 = x1
comb_featureR = Dense(units=2, name="Output_comb_feature1")(dropped1)
comb_featureR1 = Reshape([2], name='Remove_Dim1')(comb_featureR)
#only RP
onlyRP_input = Input((8,1,1))
onlyRP_inputt = Reshape([1,8],name='OnlyRP_input')(onlyRP_input)
x2 = onlyRP_inputt
x2 = Dense(units=1, activation='tanh', name="DenseRP")(x2)
hidden_layers2 = 2
for i in range(hidden_layers2-1):
    x2 = Dense(units=1, activation='tanh', name="Dense2{}".format(i))(x2)
    #x = Dropout(0.2, name='Drop{}'.format(i))(x)
dropped2 = x2
comb_feature2 = Dense(units=2, name="Output_RP_feature")(dropped2)
comb_featureR2 = Reshape([2], name='Remove_Dim2')(comb_feature2)
#only SP
onlySP_input = Input((4,1,1))
onlySP_inputt = Reshape([1,4],name='OnlySP_input')(onlySP_input)
x3 = onlySP_inputt
x3 = Dense(units=1, activation='tanh', name="DenseSP")(x3)
hidden_layers3 = 2
for i in range(hidden_layers3-1):
    x3 = Dense(units=1, activation='tanh', name="Dense3{}".format(i))(x3)
    #x = Dropout(0.2, name='Drop{}'.format(i))(x)
dropped3 = x3
comb_feature3 = Dense(units=2, name="Output_SP_feature")(dropped3)
comb_featureR3 = Reshape([2], name='Remove_Dim3')(comb_feature3)
#adding four components of utilities
final_utilities = Add(name="New_Utility_functions")([comb_featureR1,comb_featureR2, comb_featureR3])
logits = Activation('softmax', name='Choice')(final_utilities)
model = Model(inputs=[combined_input,onlyRP_input, onlySP_input], outputs=logits)
model.summary()
W11 = model.trainable_variables[6]
W12 = model.trainable_variables[7]
W13 = model.trainable_variables[12]
W14 = model.trainable_variables[13]
def custom_loss(weight11, weight12,weight13,weight14):
    def _custom_loss():
        loss = (10**(-20)) * (tf.sqrt(tf.reduce_sum(tf.square(weight11))+tf.reduce_sum(tf.square(weight12))+ tf.reduce_sum(tf.square(weight13))+tf.reduce_sum(tf.square(weight14))))
        return loss
    return _custom_loss
W21 = model.trainable_variables[10]
W22 = model.trainable_variables[11]
W23 = model.trainable_variables[16]
W24 = model.trainable_variables[17]
def custom_loss1(weight21, weight22,weight23,weight24):
    def _custom_loss1():
        loss1 = (10**(-20)) * (tf.sqrt(tf.reduce_sum(tf.square(weight21))+tf.reduce_sum(tf.square(weight22))+ tf.reduce_sum(tf.square(weight23))+tf.reduce_sum(tf.square(weight24))))
        return loss1
    return _custom_loss1
W31 = model.trainable_variables[8]
W32 = model.trainable_variables[9]
W33 = model.trainable_variables[14]
W34 = model.trainable_variables[15]
W41 = W21 - W31
W42 = W22 - W32
W43 = W23 - W33
W44 = W24 - W34
def custom_loss2(weight41,weight42,weight43,weight44):
    def _custom_loss2():
        loss2 = (10**(-20)) * (tf.sqrt(tf.reduce_sum(tf.square(weight42))+ tf.reduce_sum(tf.square(weight43))+tf.reduce_sum(tf.square(weight44))))
        return loss2
    return _custom_loss2
model.add_loss(custom_loss(W11, W12,W13,W14 ))
model.add_loss(custom_loss1(W21, W22,W23, W24))
model.add_loss(custom_loss2(W41, W42,W43, W44))

In [ ]:
import numpy as np
import tensorflow as tf
import random
import shelve
import _pickle as pickle
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, EarlyStopping
from keras.models import load_model
from keras.layers import Input, Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, Add, Reshape
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils
from keras.losses import mean_squared_error
from keras.callbacks import EarlyStopping
#choicedata
choice_data = np.load('/home/rahul/interaction2/ITTATTnterMTL/1/1/MNLneural_choice.npy')
train_labels = Reshape([2]) (choice_data)
#combineddata
combined_data = np.load('/home/rahul/interaction2/ITTATTnterMTL/1/1/MNLneural_comb.npy')
combined_data  = np.delete(combined_data, -1, axis = 2)
def normalize(data):
    return (data-data.mean(axis=0))/(data.std(axis=0))
combined_data = normalize(combined_data )
#RPdata
RP_data = np.load('/home/rahul/interaction2/ITTATTnterMTL/1/1/MNLneural_RP.npy')
RP_data = np.delete(RP_data, -1, axis = 2)
def normalize(data):
    return (data-data.mean(axis=0))/(data.std(axis=0))
RP_data = normalize(RP_data)
#SP utilities
SP_data = np.load('/home/rahul/interaction2/ITTATTnterMTL/1/1/MNLneural_SP.npy')
SP_data = np.delete(SP_data, -1, axis = 2)
def normalize(data):
    return (data-data.mean(axis=0))/(data.std(axis=0))
SP_data = normalize(SP_data)
#estimation
#callback = EarlyStopping(monitor ='loss', min_delta=0.000004, patience =15,restore_best_weights=True)
model.compile(optimizer = Adam(clipnorm = 50.), metrics = [tf.keras.metrics.CategoricalAccuracy()], loss = 'categorical_crossentropy')
history = model.fit([combined_data,RP_data,SP_data],train_labels, epochs = 2000, batch_size = 50, verbose = 2)
evaluation = model.evaluate([combined_data,RP_data,SP_data],train_labels,verbose=2)